## Predicting Car Prices

In [ ]:
import numpy as np
import pandas as pd
import plotly.express as px

from sklearn.metrics import r2_score, root_mean_squared_error
from sklearn.model_selection import train_test_split

# For getting make and model
from vininfo import Vin

from xgboost import XGBRegressor 

In [584]:
df = pd.read_csv('data/vehicles_id.csv', index_col=0)
print(df.shape)
df.columns
# df.head()

(8961, 26)


Index(['id', 'url', 'region', 'region_url', 'price', 'year', 'manufacturer',
       'model', 'condition', 'cylinders', 'fuel', 'odometer', 'title_status',
       'transmission', 'VIN', 'drive', 'size', 'type', 'paint_color',
       'image_url', 'description', 'county', 'state', 'lat', 'long',
       'posting_date'],
      dtype='object')

### Fix cylinders

In [585]:
pd.set_option('display.max_rows', 100)

# df = df[df['cylinders'] != 'other']
# df['cylinders_count'] = df['cylinders'].str.extract(r'(\d+)').astype(float)
# median_cylinders = df['cylinders_count'].median()
# df['cylinders'] = df['cylinders_count'].fillna(median_cylinders).astype(int)

df['cylinders'].value_counts()

cylinders
4 cylinders     1804
8 cylinders     1732
6 cylinders     1647
5 cylinders       19
10 cylinders      15
other             15
3 cylinders        5
Name: count, dtype: int64

### Standardize

In [586]:
def standardizeUsingDict(columnValue, allowedValues):
    temp = columnValue
    try:
        if allowedValues[columnValue]:
            temp=allowedValues[columnValue]
        return temp
    except:
        return temp
    
def standardizeManufacturer(manufacturer):
    company={'aston':'aston-martin', 'chev':'chevrolet','harley':'harley-davidson', 'land rover' :'landrover', 
            'mercedes':'mercedes-benz', 'vw': 'volkswagen', 'alfa':'alfa-romeo', 'mercedesbenz':'mercedes-benz'}
    return standardizeUsingDict(manufacturer, company)
    

def standardizeDrive(drive):
    company={'awd':'4wd', '4x4':'4wd'}
    return standardizeUsingDict(drive, company)

### Find missing values

In [587]:
# find missing values in dataset
missing = df.isnull().sum()
missing = missing[missing > 0]

missing.sort_values(inplace=True)
missing

description        1
fuel               5
long              17
lat               17
transmission      18
odometer          23
title_status      59
model             75
manufacturer     242
type            1407
drive           1639
paint_color     2188
VIN             2582
cylinders       3724
condition       5722
size            6591
county          8961
dtype: int64

In [588]:
# Estimate milage readings for missing values
def odometerReading(year):
    return 15000* (2018-year)

df.loc[df['odometer'].isna() | (df['odometer'] < 100), 'odometer'] = df.loc[df['odometer'].isna() | (df['odometer'] < 100), 'year'].apply(odometerReading)


# Set all missing paint_color values to 'white' since that is the most common car color in Idaho https://www.edmunds.com/most-popular-car-colors/
df['paint_color'] = df['paint_color'].fillna('white')

### Imputing with the VIN

In [589]:
print('Before')
print(f"Missing manufacturer count: {df['manufacturer'].isna().sum()}")
print(f"Missing model count: {df['model'].isna().sum()}")

df['VIN'] = df['VIN'].fillna('')

def fill_missing_from_vin(row):
    if row['VIN']:
        try:
            vin = Vin(row['VIN'])
            if pd.isna(row['manufacturer']):
                row['manufacturer'] = vin.manufacturer
            if pd.isna(row['model']):
                if vin.details and vin.details.model and vin.details.model.name:
                    row['model'] = vin.details.model.name
        except Exception as e:
            print(f'{e}; VIN="{row['VIN']}"')
    return row

missing_rows = df[df['manufacturer'].isna() | df['model'].isna()]
df.loc[missing_rows.index] = missing_rows.apply(fill_missing_from_vin, axis=1)

print('After')
print(f"Missing manufacturer count: {df['manufacturer'].isna().sum()}")
print(f"Missing model count: {df['model'].isna().sum()}")

df = df.drop(columns=['VIN'], axis=1)

Before
Missing manufacturer count: 242
Missing model count: 75
VIN number requires 17 chars (7 given); VIN="1412590"
VIN number requires 17 chars (13 given); VIN="3J57C7M216225"
VIN number requires 17 chars (10 given); VIN="002A207783"
VIN number requires 17 chars (8 given); VIN="E5135225"
VIN number requires 17 chars (7 given); VIN="G24I804"
VIN number requires 17 chars (13 given); VIN="BH29F8B180602"
VIN number requires 17 chars (13 given); VIN="3386771102649"
VIN number requires 17 chars (7 given); VIN="G24I804"
VIN number requires 17 chars (13 given); VIN="336679M370132"
VIN number requires 17 chars (9 given); VIN="508C17202"
After
Missing manufacturer count: 161
Missing model count: 75


### Pulling meaning from the description

In [590]:
pd.set_option('display.max_colwidth', None)
df['description'] = df['description'].str.strip()
df['description'] = df['description'].fillna('')

# Define keyword lists
accident_keywords = ['accident', 'wreck', 'crash', 'damaged', 'repair', 'reconstructed', 'totaled', 'rebuilt', 'branded title']
no_accident_phrases = ['no accidents', 'accident free', 'clean title', 'no damage', 'no accident', 'never wrecked']
flood_keywords = ['flood', 'water damage', 'submerged', 'hurricane damage']
salvage_keywords = ['salvage', 'rebuilt', 'branded title', 'totaled', 'irreparable']

accident_pattern = r'\b(' + '|'.join(accident_keywords) + r')\b'
no_accident_pattern = r'\b(' + '|'.join(no_accident_phrases) + r')\b'
flood_pattern = '|'.join(flood_keywords)
salvage_pattern = '|'.join(salvage_keywords)

# Flag descriptions containing keywords
df['no_accident'] = df['description'].str.contains(no_accident_pattern, case=False, regex=True)
df['has_accident'] = df['description'].str.contains(accident_pattern, case=False, regex=True)
df['has_flood'] = df['description'].str.contains(flood_pattern, case=False, regex=True)
df['has_salvage'] = df['description'].str.contains(salvage_pattern, case=False, regex=True)

# Negate the no_accidents in the has_accidents (with bitwise operations)
df['has_accident'] = df['has_accident'] & (~df['no_accident'])

# Display the DataFrame with the new flags
print(df[['description', 'no_accident', 'has_accident', 'has_flood', 'has_salvage']].value_counts())

matching_rows = df[df['has_accident'] == True]
for i in range(3):
    print(f"Description {i+1}: {matching_rows['description'].iloc[i]}")

df = df.drop(columns=['description'], axis=1)

C:\Users\leviw\AppData\Local\Temp\ipykernel_27540\1635215651.py:17: UserWarning:

This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.

C:\Users\leviw\AppData\Local\Temp\ipykernel_27540\1635215651.py:18: UserWarning:

This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.



description                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             

In [591]:
# Function to calculate mean and median prices for a given flag
def calculate_stats(flag):
    mean_prices = df.groupby(flag)['price'].mean().reset_index(name='mean_price')
    median_prices = df.groupby(flag)['price'].median().reset_index(name='median_price')
    stats_df = pd.merge(mean_prices, median_prices, on=flag)
    stats_df['condition'] = flag
    # Rename the flag column to a common name
    stats_df = stats_df.rename(columns={flag: 'flag_value'})
    return stats_df

# Calculate stats for each condition indicator
accident_stats = calculate_stats('has_accident')
flood_stats = calculate_stats('has_flood')
salvage_stats = calculate_stats('has_salvage')

# Combine all stats into one DataFrame
all_stats = pd.concat([accident_stats, flood_stats, salvage_stats], ignore_index=True)

# Melt the DataFrame for plotting
melted_stats = all_stats.melt(
    id_vars=['condition', 'flag_value'],
    value_vars=['mean_price', 'median_price'],
    var_name='statistic',
    value_name='price'
)

# Convert flag_value to string for consistent plotting
melted_stats['flag_value'] = melted_stats['flag_value'].astype(str)

# Plot the data using Plotly Express
fig = px.bar(
    melted_stats,
    x='flag_value',
    y='price',
    color='statistic',
    barmode='group',
    facet_col='condition',
    category_orders={'flag_value': ['False', 'True']},
    title='Mean and Median Price Based on Info From Description',
    labels={'flag_value': 'Condition Present', 'price': 'Price', 'condition': 'Condition Indicator'}
)

fig.update_layout(
    yaxis_title='Price',
    legend_title='Statistic'
)

# Update facet titles to show only the condition name
fig.for_each_annotation(lambda a: a.update(text=a.text.split('=')[1]))

fig.show()

### Drop unneeded columns

In [592]:
# County is always blank. The rest of these are unusable
df = df.drop(['id', 'url', 'region_url', 'image_url', 'state', 'county'], axis=1)
df.columns

Index(['region', 'price', 'year', 'manufacturer', 'model', 'condition',
       'cylinders', 'fuel', 'odometer', 'title_status', 'transmission',
       'drive', 'size', 'type', 'paint_color', 'lat', 'long', 'posting_date',
       'no_accident', 'has_accident', 'has_flood', 'has_salvage'],
      dtype='object')

### One-hot encoding

In [593]:
# Convert posting_date to datetime
df['posting_date'] = pd.to_datetime(
    df['posting_date'].str.strip(),
    format='%Y-%m-%dT%H:%M:%S%z',
    errors='coerce',
    utc=True
)

df['posting_year'] = df['posting_date'].dt.year
df['posting_month'] = df['posting_date'].dt.month
df['posting_day'] = df['posting_date'].dt.day

df = df.drop(columns=['posting_date'])

categorical_columns = [
    'region', 'manufacturer', 'model', 'condition',
    'fuel', 'title_status', 'transmission', 'drive', 
    'type', 'paint_color',  'size', 'cylinders'
]
df = pd.get_dummies(df, columns=categorical_columns)

### Graph time

In [594]:
fig = px.box(df, y='price', title='Price Distribution with Outliers')
fig.update_layout(yaxis_title='Price')
fig.show()

### Remove outliers

In [595]:
df = df.drop(df[(df['price'] > 100000) | (df['price'] < 1000)].index)

In [596]:
fig = px.box(df, y='price', title='Price Distribution without Outliers')
fig.update_layout(yaxis_title='Price')
fig.show()

In [597]:
# Drop rows with missing values in 'cylinders_count', 'size', or 'price'
# df_clean = df.dropna(subset=['cylinders', 'price'])

# # Create a box plot
# fig = px.box(df_clean, x='cylinders_count', y='price', title='Price by Cylinder Count')
# fig.update_layout(
#     xaxis_title='Cylinder Count',
#     yaxis_title='Price'
# )
# fig.show()

### Create the model

In [598]:
X = df.drop(['price'], axis=1)
y = df['price']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2, random_state=41)

model = XGBRegressor(objective ='reg:squarederror')
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

### Stop. Evaluate

In [600]:
# Print rmse and r2
rmse = root_mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print(f'RMSE: {rmse}')
print(f'R^2: {r2}')

# RMSE: 9503.794199710273
# R^2: 0.7246636152267456

# With outliers removed
# RMSE: 5880.276820217292
# R^2: 0.8839418888092041

# RMSE: 5664.881271275077
# R^2: 0.8922886252403259

# RMSE: 5410.1818027355575
# R^2: 0.8976122140884399

RMSE: 5414.786215534579
R^2: 0.8974378705024719
